In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector as selector
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import set_config
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold

In [4]:
set_config(display='diagram')

In [5]:
trainData = 'train.csv'
testData = 'test.csv'
index_col = 'Id'
target_col = 'SalePrice'

In [6]:
def getMissingColumns(df):
    """ Return columns containing missing data """
    return df.columns[df.isna().any()]

In [7]:
def summariseMissing(df):
    
    NcolsNA = df.isna().any(axis=0).sum()
    propColsNA = NcolsNA / len(df.columns)
    NrowsNA = df.isna().any(axis=1).sum()
    propRowsNA = NrowsNA / len(df)
    totalNA = df.isna().to_numpy().sum()
    propTotalNA = totalNA / df.to_numpy().size
    # Get largest integer width for formatting
    w = max([len(str(x)) for x in [NcolsNA, NrowsNA, totalNA]])
    print(f'{"Columns with missing data:":26}{NcolsNA:{w}}\t{propColsNA:6.1%}')
    print(f'{"Rows with missing data:":26}{NrowsNA:{w}}\t{propRowsNA:6.1%}')
    print(f'{"Total missing data:":26}{totalNA:{w}}\t{propTotalNA:6.1%}')
    
    # Return number of missing datapoints per column
    missingCount = df.isna().sum()
    return missingCount[missingCount > 0]

In [8]:
def getCategorical(df, maxUnique=None):
    
    catDf = df.select_dtypes(include=['object', 'category'])
    if maxUnique is not None:
        catCols = catDf.columns[catDf.nunique() <= maxUnique]
    else:
        catCols = catDf.columns
    return list(catCols)

In [9]:
X = pd.read_csv(trainData, index_col=index_col)
y = X.pop(target_col)

In [205]:
X_test = pd.read_csv(testData, index_col=index_col)

In [206]:
split = train_test_split(X, y, random_state=0, train_size=0.8, test_size=0.2)
X_train, X_valid, y_train, y_valid = map(lambda x: x.copy(), split)

In [207]:
catCols = getCategorical(X_train, maxUnique=900)

In [208]:
numberCols = list(X_train.select_dtypes(include=['number']).columns)

In [209]:
X = X[catCols + numberCols]
X_test = X_test[catCols + numberCols]

In [242]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  RobustScaler()),
    ('power',   PowerTransformer(method='yeo-johnson'))])

In [275]:
categorical_transformer = Pipeline(steps=[
    ('encode' , OrdinalEncoder()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot' , OneHotEncoder(handle_unknown='ignore'))])

In [276]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_include='number')),
    ('cat', categorical_transformer, selector(dtype_include=['category', 'object']))])

In [277]:
model = XGBRegressor(n_jobs=4, random_state=0)

In [278]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',   model)])

In [279]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5, 7, 10],
    'classifier__min_child_weight': [1, 3, 5],
    'classifier__subsample': [0.5, 0.7],
    'classifier__colsample_bytree': [0.5, 0.7],
    'classifier__n_estimators' : [100, 200, 500],
    'classifier__objective': ['reg:squarederror']
    }

param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__n_estimators': [100, 500, 1000],
    'classifier__learning_rate': [0.01, 0.05, 0.1]
}

In [280]:
# Configure the cross-validation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=1)

In [281]:
grid_search = GridSearchCV(clf, param_grid, cv=cv, refit=True)

In [282]:
# Apply transformation to the target
model = TransformedTargetRegressor(regressor=grid_search, transformer=PowerTransformer(method='yeo-johnson'))

In [283]:
model.fit(X, y)

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

...

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/joblib/memory.py", line 355, in __call__
    return self.func(*args, **kwargs)
  File "/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transfo

ValueError: Input contains NaN

In [252]:
print(model.regressor_.best_score_)
print(f'Best params:', model.regressor_.best_params_)

0.8795674898115019
Best params: {'classifier__learning_rate': 0.05, 'classifier__n_estimators': 1000, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'median'}


In [240]:
preds = model.predict(X_test)
preds

array([127240.25138787, 152115.7061867 , 176231.70953744, ...,
       154090.8914048 , 113410.27824231, 229801.46012747])

In [269]:
from sklearn.preprocessing import OrdinalEncoder
test = np.array(['9,', '10']).reshape(-1, 1)

enc = OrdinalEncoder()
simImputer = SimpleImputer()
enc.fit(test)

OrdinalEncoder()

In [284]:
encoded = enc.transform(np.array(['9,', '10', np.nan]).reshape(-1, 1))
simImputer.fit(encoded)

ValueError: Found unknown categories ['nan'] in column 0 during transform

In [286]:
ordinal_cols_mapping = [{
    "col":"ExterQual",    
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.nan)
    ]},
]
encoder = OrdinalEncoder(mapping = ordinal_cols_mapping, 
                         return_df = True)  

TypeError: __init__() got an unexpected keyword argument 'mapping'

In [298]:
encoder = OrdinalEncoder(categories=[["bad", "average", "good"]])
encoder.fit_transform([["good"], ["bad"], ["bad"], ["average"]], [np.nan])

array([[2.],
       [0.],
       [0.],
       [1.]])

In [ ]:
# Create transformer pipeline for normal categories
# Create transformer pipeline for ordinal catgories
# Create transform for cyclical transformation of months
# Numerical transformer for skewed
# Numerical transformer for normal data